<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PostgreSQL_03A_ROLAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Yantrajaal Banner](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install & Configure Postgres

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2024-01-05 05:46:40.645692+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


For Connecting to a Postgres server on Amazon RDS, see [this notebook](https://github.com/Praxis-QR/RDWH/blob/main/PostgreSQL_01_Install_Config.ipynb)

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#!sudo -u postgres psql -V
!psql -V
!service postgresql start





psql (PostgreSQL) 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
#!pip install psycopg2
import psycopg2
import pandas as pd

In [4]:
# This has created an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dbpraxis and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dbpraxis


ALTER ROLE
ALTER ROLE


In [5]:
# Create file to store the password. Else will be prompted each time
#!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dbpraxis:upraxis:upass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:dbpraxis:upraxis:upass


## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [6]:
# Local Colab server

conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
#Executing an MYSQL function using the execute() method
cursor.execute("select version()")
# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
#Closing the connection
conn.close()

Connection established to:  ('PostgreSQL 14.10 (Ubuntu 14.10-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [7]:
# function to execute SQL select statements with Postgres
#
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return
# function to execute SQL non-select statements with Postgres
#
def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dbpraxis", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

# Build Standard Fact Table

## Create Normalised Fact Table

In [8]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
DROP table if exists CarSalesFact_N;\
CREATE TABLE if not exists CarSalesFact_N ( Txn int ,  Dim  varchar(8) ,  DimValue varchar(8) );\
INSERT INTO CarSalesFact_N (Txn, Dim, DimValue) VALUES \
 (1, 'Make', 'Honda'),(1, 'Type', 'SUV'),(1, 'Colour', 'Red'),\
 (2, 'Make', 'Honda'),(2, 'Type', 'SUV'),(2, 'Colour', 'Black'), \
 (3, 'Make', 'Honda'),(3, 'Type', 'Sedan'),(3, 'Colour', 'Brown'),\
 (4, 'Make', 'Toyota'),(4, 'Type', 'Compact'),(4, 'Colour', 'Red'),\
 (5, 'Make', 'Toyota'),(5, 'Colour', 'Black'),(5, 'Type', 'Sedan'),\
 (6, 'Make', 'Toyota'),(6, 'Colour', 'Black'),(6, 'Type', 'Sedan'),\
 (7, 'Make', 'Honda'),(7, 'Type', 'SUV'),(7, 'Colour', 'Black'),\
 (8, 'Make', 'Honda'),(8, 'Type', 'SUV'),(8, 'Colour', 'Red'),\
 (9, 'Make', 'Honda'),(9, 'Type', 'SUV'),(9, 'Colour', 'Black'),\
 (10, 'Make', 'Honda'),(10, 'Type', 'SUV'),(10, 'Colour', 'Black'),\
 (11, 'Make', 'Mazda'),(11, 'Type', 'SUV'),(11, 'Colour', 'Black'),\
 (12, 'Make', 'Mazda'),(12, 'Type', 'Compact'),(12, 'Colour', 'Red'),\
 (13, 'Make', 'Mazda'),(13, 'Type', 'Sedan'),(13, 'Colour', 'Brown'),\
 (14, 'Make', 'Mazda'),(14, 'Type', 'SUV'),(14, 'Colour', 'Red');\
"
#runCmd(ddl)


NOTICE:  table "carsalesfact_n" does not exist, skipping
INSERT 0 42


## Transpose ("Pivot") Columns to Rows
Convert a Normalised Fact Table to a Standard Fact Table

In [9]:
#runSelect("select * from CarSalesFact_N")
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from CarSalesFact_N"

 txn |  dim   | dimvalue 
-----+--------+----------
   1 | Make   | Honda
   1 | Type   | SUV
   1 | Colour | Red
   2 | Make   | Honda
   2 | Type   | SUV
   2 | Colour | Black
   3 | Make   | Honda
   3 | Type   | Sedan
   3 | Colour | Brown
   4 | Make   | Toyota
   4 | Type   | Compact
   4 | Colour | Red
   5 | Make   | Toyota
   5 | Colour | Black
   5 | Type   | Sedan
   6 | Make   | Toyota
   6 | Colour | Black
   6 | Type   | Sedan
   7 | Make   | Honda
   7 | Type   | SUV
   7 | Colour | Black
   8 | Make   | Honda
   8 | Type   | SUV
   8 | Colour | Red
   9 | Make   | Honda
   9 | Type   | SUV
   9 | Colour | Black
  10 | Make   | Honda
  10 | Type   | SUV
  10 | Colour | Black
  11 | Make   | Mazda
  11 | Type   | SUV
  11 | Colour | Black
  12 | Make   | Mazda
  12 | Type   | Compact
  12 | Colour | Red
  13 | Make   | Mazda
  13 | Type   | Sedan
  13 | Colour | Brown
  14 | Make   | Mazda
  14 | Type   | SUV
  14 | Colour | Red
(42 rows)



In [10]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c " \
SELECT \
  Txn,\
  string_agg(case when Dim = 'Make' then DimValue else NULL end,';') Make,\
  string_agg(case when Dim = 'Type' then DimValue else NULL end,';') Type,\
  string_agg(case when Dim = 'Colour' then DimValue else NULL end,';') Colour \
FROM CarSalesFact_N \
GROUP BY txn\
"
#runSelect(sql)


 txn |  make  |  type   | colour 
-----+--------+---------+--------
   1 | Honda  | SUV     | Red
   2 | Honda  | SUV     | Black
   3 | Honda  | Sedan   | Brown
   4 | Toyota | Compact | Red
   5 | Toyota | Sedan   | Black
   6 | Toyota | Sedan   | Black
   7 | Honda  | SUV     | Black
   8 | Honda  | SUV     | Red
   9 | Honda  | SUV     | Black
  10 | Honda  | SUV     | Black
  11 | Mazda  | SUV     | Black
  12 | Mazda  | Compact | Red
  13 | Mazda  | Sedan   | Brown
  14 | Mazda  | SUV     | Red
(14 rows)



In [11]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c " \
select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
    ( \
    SELECT \
    Txn,\
    string_agg(case when Dim = 'Make' then DimValue else NULL end,';') Make,\
    string_agg(case when Dim = 'Type' then DimValue else NULL end,';') Type,\
    string_agg(case when Dim = 'Colour' then DimValue else NULL end,';') Colour \
    FROM CarSalesFact_N \
    GROUP BY txn\
    ) as csf0 \
group by \
csf0.Make, csf0.Type, csf0.Colour \
"
#runSelect(sql)

  make  |  type   | colour | units 
--------+---------+--------+-------
 Honda  | SUV     | Red    |     2
 Honda  | SUV     | Black  |     4
 Toyota | Compact | Red    |     1
 Mazda  | Compact | Red    |     1
 Mazda  | SUV     | Red    |     1
 Toyota | Sedan   | Black  |     2
 Honda  | Sedan   | Brown  |     1
 Mazda  | Sedan   | Brown  |     1
 Mazda  | SUV     | Black  |     1
(9 rows)



## Create Standard Fact Table

In [12]:
ddl = "\
drop table if exists CarSalesFact_S; \
create table if not exists CarSalesFact_S as ( \
    select csf0.Make, csf0.Type, csf0.Colour, count(txn) Units from \
        ( \
        SELECT \
        Txn,\
        string_agg(case when Dim = 'Make' then DimValue else NULL end,';') Make,\
        string_agg(case when Dim = 'Type' then DimValue else NULL end,';') Type,\
        string_agg(case when Dim = 'Colour' then DimValue else NULL end,';') Colour \
        FROM CarSalesFact_N \
        GROUP BY txn\
        ) as csf0 \
    group by \
    csf0.Make, csf0.Type, csf0.Colour \
    ) \
"
runCmd(ddl)

Command executed


In [13]:
#runSelect("select * from CarSalesFact_S")
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "select * from CarSalesFact_S"

  make  |  type   | colour | units 
--------+---------+--------+-------
 Honda  | SUV     | Red    |     2
 Honda  | SUV     | Black  |     4
 Toyota | Compact | Red    |     1
 Mazda  | Compact | Red    |     1
 Mazda  | SUV     | Red    |     1
 Toyota | Sedan   | Black  |     2
 Honda  | Sedan   | Brown  |     1
 Mazda  | Sedan   | Brown  |     1
 Mazda  | SUV     | Black  |     1
(9 rows)



#Cross Tab a.k.a Pivot

## Level 1 - Simple Group By

In [14]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select Make, Type, Sum(Units) Count \
  from CarSalesFact_S \
  group by Make, Type \
"
#runSelect(sql)

  make  |  type   | count 
--------+---------+-------
 Mazda  | SUV     |     2
 Honda  | SUV     |     6
 Mazda  | Sedan   |     1
 Toyota | Compact |     1
 Toyota | Sedan   |     2
 Honda  | Sedan   |     1
 Mazda  | Compact |     1
(7 rows)



## Level 2 : Group BY with ROLLUP

### Basic Rollup

In [15]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c"\
select Make, Type, Sum(Units) Count \
  from CarSalesFact_S \
  group by rollup(Make, Type) \
"
#runSelect(sql)

  make  |  type   | count 
--------+---------+-------
        |         |    14
 Mazda  | SUV     |     2
 Honda  | SUV     |     6
 Mazda  | Sedan   |     1
 Toyota | Compact |     1
 Toyota | Sedan   |     2
 Honda  | Sedan   |     1
 Mazda  | Compact |     1
 Honda  |         |     7
 Toyota |         |     3
 Mazda  |         |     4
(11 rows)



In [16]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select \
    COALESCE(Make, 'All Makes') AS Make, \
    COALESCE(Type, 'All Types') AS Type, \
    Sum(Units) Count \
  from CarSalesFact_S \
  group by rollup(Make, Type) \
"
#runSelect(sql)

   make    |   type    | count 
-----------+-----------+-------
 All Makes | All Types |    14
 Mazda     | SUV       |     2
 Honda     | SUV       |     6
 Mazda     | Sedan     |     1
 Toyota    | Compact   |     1
 Toyota    | Sedan     |     2
 Honda     | Sedan     |     1
 Mazda     | Compact   |     1
 Honda     | All Types |     7
 Toyota    | All Types |     3
 Mazda     | All Types |     4
(11 rows)



### Roll Up More

In [17]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select \
    COALESCE(Make, 'All Makes') AS Make, \
    Sum(Units) Count \
  from CarSalesFact_S \
  group by rollup(Make) \
"
#runSelect(sql)

   make    | count 
-----------+-------
 All Makes |    14
 Honda     |     7
 Toyota    |     3
 Mazda     |     4
(4 rows)



In [18]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select \
    COALESCE(Type, 'All Types') AS Type, \
    Sum(Units) Count \
  from CarSalesFact_S \
  group by rollup(Type) \
"
#runSelect(sql)

   type    | count 
-----------+-------
 All Types |    14
 SUV       |     8
 Sedan     |     4
 Compact   |     2
(4 rows)



### Roll Up - Even More

In [19]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select \
    Sum(Units) Count \
  from CarSalesFact_S \
"
#runSelect(sql)

 count 
-------
    14
(1 row)



## Level 3 - Full Crosstab

### "Jugaad" Crosstab

In [33]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select Make, Type, Sum(Units) Count \
    from CarSalesFact_S \
    group by Make, Type \
union all \
SELECT Make, 'ALL', SUM(Units) \
    FROM CarSalesFact_S \
    GROUP BY Make \
union all \
SELECT 'ALL', Type, SUM(Units) \
    FROM CarSalesFact_S \
    GROUP BY Type \
union all \
SELECT 'ALL', 'ALL', SUM(Units) \
    FROM CarSalesFact_S \
"

  make  |  type   | count 
--------+---------+-------
 Mazda  | SUV     |     2
 Honda  | SUV     |     6
 Mazda  | Sedan   |     1
 Toyota | Compact |     1
 Toyota | Sedan   |     2
 Honda  | Sedan   |     1
 Mazda  | Compact |     1
 Honda  | ALL     |     7
 Toyota | ALL     |     3
 Mazda  | ALL     |     4
 ALL    | SUV     |     8
 ALL    | Sedan   |     4
 ALL    | Compact |     2
 ALL    | ALL     |    14
(14 rows)



### CUBE solution

In [35]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
select \
    COALESCE(Make, 'All Makes') AS Make, \
    COALESCE(Type, 'All Types') AS Type, \
    Sum(Units) Count \
  from CarSalesFact_S \
  group by cube(Make, Type) \
"

   make    |   type    | count 
-----------+-----------+-------
 All Makes | All Types |    14
 Mazda     | SUV       |     2
 Honda     | SUV       |     6
 Mazda     | Sedan     |     1
 Toyota    | Compact   |     1
 Toyota    | Sedan     |     2
 Honda     | Sedan     |     1
 Mazda     | Compact   |     1
 Honda     | All Types |     7
 Toyota    | All Types |     3
 Mazda     | All Types |     4
 All Makes | SUV       |     8
 All Makes | Sedan     |     4
 All Makes | Compact   |     2
(14 rows)



### "True" Crosstab

In [36]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c "\
SELECT \
	Make,\
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S\
  GROUP BY ROLLUP (Make)\
"
#runSelect(sql)

  make  | sedan | suv | compact 
--------+-------+-----+---------
        |     4 |   8 |       2
 Honda  |     1 |   6 |       0
 Toyota |     2 |   0 |       1
 Mazda  |     1 |   2 |       1
(4 rows)



In [37]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c"\
SELECT \
    COALESCE(Make, 'Total') AS Dim, \
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S\
  GROUP BY ROLLUP (Make)\
"
#runSelect(sql)

  dim   | sedan | suv | compact 
--------+-------+-----+---------
 Total  |     4 |   8 |       2
 Honda  |     1 |   6 |       0
 Toyota |     2 |   0 |       1
 Mazda  |     1 |   2 |       1
(4 rows)



In [38]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c  "\
SELECT \
COALESCE(Make, 'totals') AS Dim, \
sums.SUV, sums.Sedan, sums.Compact,\
sums.SUV + sums.Sedan + sums.Compact  Totals \
FROM ( \
  SELECT \
    Make, \
	sum(case when (Type = 'Sedan') then Units else 0 end) as Sedan,\
	sum(case when (Type = 'SUV') then Units else 0 end) as SUV,\
	sum(case when (Type = 'Compact') then Units else 0 end) as Compact\
  FROM CarSalesFact_S\
  GROUP BY ROLLUP (Make)\
) AS sums \
"
#runSelect(sql)

  dim   | suv | sedan | compact | totals 
--------+-----+-------+---------+--------
 totals |   8 |     4 |       2 |     14
 Honda  |   6 |     1 |       0 |      7
 Toyota |   0 |     2 |       1 |      3
 Mazda  |   2 |     1 |       1 |      4
(4 rows)



In [23]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c  "\
SELECT \
COALESCE(Colour, 'totals') AS Dim, \
sums.Honda, sums.Toyota, sums.Mazda,\
sums.Honda + sums.Toyota + sums.Mazda  Totals \
FROM ( \
  SELECT \
    Colour, \
	sum(case when (Make = 'Honda') then Units else 0 end) as Honda,\
	sum(case when (Make = 'Toyota') then Units else 0 end) as Toyota,\
	sum(case when (Make = 'Mazda') then Units else 0 end) as Mazda\
  FROM CarSalesFact_S\
  GROUP BY ROLLUP (Colour)\
) AS sums \
"

  dim   | honda | toyota | mazda | totals 
--------+-------+--------+-------+--------
 totals |     7 |      3 |     4 |     14
 Red    |     2 |      1 |     2 |      5
 Black  |     4 |      2 |     1 |      7
 Brown  |     1 |      0 |     1 |      2
(4 rows)



In [24]:
!psql -h localhost -p 5432 -Uupraxis -ddbpraxis -c  "\
SELECT \
COALESCE(Type, 'totals') AS Dim, \
sums.Red, sums.Black, sums.Brown,\
sums.Red + sums.Black + sums.Brown  Totals \
FROM ( \
  SELECT \
    Type, \
	sum(case when (Colour = 'Red') then Units else 0 end) as Red,\
	sum(case when (Colour = 'Black') then Units else 0 end) as Black,\
	sum(case when (Colour = 'Brown') then Units else 0 end) as Brown\
  FROM CarSalesFact_S\
  GROUP BY ROLLUP (Type)\
) AS sums \
"

   dim   | red | black | brown | totals 
---------+-----+-------+-------+--------
 totals  |   5 |     7 |     2 |     14
 SUV     |   3 |     5 |     0 |      8
 Sedan   |   0 |     2 |     2 |      4
 Compact |   2 |     0 |     0 |      2
(4 rows)



#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)